# Simulation
## Group project
### 42180 Quantitative modelling of behaviour

------

**Group 11** <br>

------

Camille Babin <br>
Martine Grangé <br>
Mikkel Johansen <br>
Kyriakos Michailidis <br>
Jiri Spitzer

------

Import libraries and enable inline plotting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
from collections import Counter
from re import search as Search
from IPython.display import display

In [2]:
%matplotlib inline

Disable warnings

In [3]:
warnings.filterwarnings('ignore')

Load data

In [4]:
data_path = r'.\data\Elisa_thesis_data_noPC.csv'

In [5]:
df = pd.read_csv(data_path)
df.head(2)

,VAR1,ID,CHOICE,Alt_Description_ICV1,CarClass_ICV1,CarType_ICV1,Purchaseprice_ICV1,Operationcost_ICV1,Yearlycost__ICV1,Range_ICV1,...,q16_20_resp,q16_21_resp,q16_22_resp,q16_23_resp,q16_24_resp,House_single,House_couple,House_children,House_couple_children,House_couple_nochildren
0,0,1,1,1,2,1,200000,1.11,4300,1097,...,1,5,4,2,5,1,0,0,0,0
1,1,1,3,1,2,1,185000,0.88,4300,898,...,1,5,4,2,5,1,0,0,0,0


#### Data preparation

Deleting rows with unknown choice.

In [6]:
df_mod=df[df['CHOICE']>0].copy(deep=True)
df_mod.reset_index();

Saving number of rows and columns.

In [7]:
n,m=df_mod.shape

In [8]:
av_cho=[]
for i in df_mod.index:
    av_cho.append([])
    if df_mod['CarClass_ICV1'].loc[i]>0: 
        av_cho[-1].append(df_mod['CarClass_ICV1'].loc[i])
    if df_mod['CarClass_ICV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_ICV2'].loc[i])
    if df_mod['CarClass_BEV1'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_BEV1'].loc[i]+6)
    if df_mod['CarClass_BEV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_BEV2'].loc[i]+6)
    if df_mod['CarClass_PHEV1'].loc[i]>0: 
        av_cho[-1].append(df_mod['CarClass_PHEV1'].loc[i]+12)
    if df_mod['CarClass_PHEV2'].loc[i]>0:
        av_cho[-1].append(df_mod['CarClass_PHEV2'].loc[i]+12)

In [9]:
avail=['AV1','AV2','AV3','AV4','AV5','AV6','AV7','AV8','AV9','AV10','AV11','AV12','AV13','AV14','AV15','AV16','AV17','AV18']
for i in range(18):
    df_mod[avail[i]]=np.zeros(n,int)
    for j in range(n):
        if i+1 in av_cho[j]:
            df_mod[avail[i]].loc[df_mod.index[j]]=1

In [10]:
df_mod['Gender_Dummy']=np.zeros(n,int)
df_mod['Dummy_Age2']=np.zeros(n,int)
df_mod['Dummy_Age3']=np.zeros(n,int)
df_mod['Dummy_Age4']=np.zeros(n,int)
df_mod['Dummy_Age5']=np.zeros(n,int)
for j in range(n):
    df_mod['Gender_Dummy'].loc[df_mod.index[j]]=df_mod['c_Gender'].loc[df_mod.index[j]]-1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==2:
        df_mod['Dummy_Age2'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==3:
        df_mod['Dummy_Age3'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==4:
        df_mod['Dummy_Age4'].loc[df_mod.index[j]]=1
    if df_mod['c_Age_cat'].loc[df_mod.index[j]]==5:
        df_mod['Dummy_Age5'].loc[df_mod.index[j]]=1

In [11]:
columns=["Size_ICV_MINI","Size_ICV_LIL","Size_ICV_MEL","Size_ICV_STOR","Size_ICV_PREM","Size_ICV_LUK",
         "Size_BEV_MINI","Size_BEV_LIL","Size_BEV_MEL","Size_BEV_STOR","Size_BEV_PREM","Size_BEV_LUK",
         "Size_PHEV_MINI","Size_PHEV_LIL","Size_PHEV_MEL","Size_PHEV_STOR","Size_PHEV_PREM","Size_PHEV_LUK"]
for column in columns:
    df_mod[column+"_Large_Dummy"]=np.zeros(n,int)
    for j in range(n):
        if df_mod[column].loc[df_mod.index[j]]>2:
            df_mod[column+"_Large_Dummy"].loc[df_mod.index[j]]=1
        else:
            df_mod[column+"_Large_Dummy"].loc[df_mod.index[j]]=0

In [12]:
pd.options.display.max_columns = 500

In [13]:
df_mod.head(5)

,VAR1,ID,CHOICE,Alt_Description_ICV1,CarClass_ICV1,CarType_ICV1,Purchaseprice_ICV1,Operationcost_ICV1,Yearlycost__ICV1,Range_ICV1,Acceleration_ICV1,Size_ICV1,CO2_ICV1,Charginginfrastructure_ICV1,Chargingspeed_ICV1,Homedistnearestcharger_ICV1,Homechargeravailability_ICV1,Alt_Description_ICV2,CarClass_ICV2,CarType_ICV2,Purchaseprice_ICV2,Operationcost_ICV2,Yearlycost__ICV2,Range_ICV2,Acceleration_ICV2,Size_ICV2,CO2_ICV2,Charginginfrastructure_ICV2,Chargingspeed_ICV2,Homedistnearestcharger_ICV2,Homechargeravailability_ICV2,Alt_Description_BEV1,CarClass_BEV1,CarType_BEV1,Purchaseprice_BEV1,Operationcost_BEV1,Yearlycost__BEV1,Range_BEV1,Acceleration_BEV1,Size_BEV1,CO2_BEV1,Charginginfrastructure_BEV1,Chargingspeed_BEV1,Homedistnearestcharger_BEV1,Homechargeravailability_BEV1,Alt_Description_BEV2,CarClass_BEV2,CarType_BEV2,Purchaseprice_BEV2,Operationcost_BEV2,Yearlycost__BEV2,Range_BEV2,Acceleration_BEV2,Size_BEV2,CO2_BEV2,Charginginfrastructure_BEV2,Chargingspeed_BEV2,Homedistnearestcharger_BEV2,Homechargeravailability_BEV2,Alt_Description_PHEV1,CarClass_PHEV1,CarType_PHEV1,Purchaseprice_PHEV1,Operationcost_PHEV1,Yearlycost__PHEV1,Range_PHEV1,Acceleration_PHEV1,Size_PHEV1,CO2_PHEV1,Charginginfrastructure_PHEV1,Chargingspeed_PHEV1,Homedistnearestcharger_PHEV1,Homechargeravailability_PHEV1,Alt_Description_PHEV2,CarClass_PHEV2,CarType_PHEV2,Purchaseprice_PHEV2,Operationcost_PHEV2,Yearlycost__PHEV2,Range_PHEV2,Acceleration_PHEV2,Size_PHEV2,CO2_PHEV2,Charginginfrastructure_PHEV2,Chargingspeed_PHEV2,Homedistnearestcharger_PHEV2,Homechargeravailability_PHEV2,Purchaseprice_ICV_MINI,Operationcost_ICV_MINI,Yearlycost__ICV_MINI,Range_ICV_MINI,Acceleration_ICV_MINI,Size_ICV_MINI,CO2_ICV_MINI,Charginginfrastructure_ICV_MINI,Chargingspeed_ICV_MINI,Homedistnearestcharger_ICV_MINI,Homechargeravailability_ICV_MINI,Purchaseprice_ICV_LIL,Operationcost_ICV_LIL,Yearlycost__ICV_LIL,Range_ICV_LIL,Acceleration_ICV_LIL,Size_ICV_LIL,CO2_ICV_LIL,Charginginfrastructure_ICV_LIL,Chargingspeed_ICV_LIL,Homedistnearestcharger_ICV_LIL,Homechargeravailability_ICV_LIL,Purchaseprice_ICV_MEL,Operationcost_ICV_MEL,Yearlycost__ICV_MEL,Range_ICV_MEL,Acceleration_ICV_MEL,Size_ICV_MEL,CO2_ICV_MEL,Charginginfrastructure_ICV_MEL,Chargingspeed_ICV_MEL,Homedistnearestcharger_ICV_MEL,Homechargeravailability_ICV_MEL,Purchaseprice_ICV_STOR,Operationcost_ICV_STOR,Yearlycost__ICV_STOR,Range_ICV_STOR,Acceleration_ICV_STOR,Size_ICV_STOR,CO2_ICV_STOR,Charginginfrastructure_ICV_STOR,Chargingspeed_ICV_STOR,Homedistnearestcharger_ICV_STOR,Homechargeravailability_ICV_STOR,Purchaseprice_ICV_PREM,Operationcost_ICV_PREM,Yearlycost__ICV_PREM,Range_ICV_PREM,Acceleration_ICV_PREM,Size_ICV_PREM,CO2_ICV_PREM,Charginginfrastructure_ICV_PREM,Chargingspeed_ICV_PREM,Homedistnearestcharger_ICV_PREM,Homechargeravailability_ICV_PREM,Purchaseprice_ICV_LUK,Operationcost_ICV_LUK,Yearlycost__ICV_LUK,Range_ICV_LUK,Acceleration_ICV_LUK,Size_ICV_LUK,CO2_ICV_LUK,Charginginfrastructure_ICV_LUK,Chargingspeed_ICV_LUK,Homedistnearestcharger_ICV_LUK,Homechargeravailability_ICV_LUK,Purchaseprice_BEV_MINI,Operationcost_BEV_MINI,Yearlycost__BEV_MINI,Range_BEV_MINI,Acceleration_BEV_MINI,Size_BEV_MINI,CO2_BEV_MINI,Charginginfrastructure_BEV_MINI,Chargingspeed_BEV_MINI,Homedistnearestcharger_BEV_MINI,Homechargeravailability_BEV_MINI,Purchaseprice_BEV_LIL,Operationcost_BEV_LIL,Yearlycost__BEV_LIL,Range_BEV_LIL,Acceleration_BEV_LIL,Size_BEV_LIL,CO2_BEV_LIL,Charginginfrastructure_BEV_LIL,Chargingspeed_BEV_LIL,Homedistnearestcharger_BEV_LIL,Homechargeravailability_BEV_LIL,Purchaseprice_BEV_MEL,Operationcost_BEV_MEL,Yearlycost__BEV_MEL,Range_BEV_MEL,Acceleration_BEV_MEL,Size_BEV_MEL,CO2_BEV_MEL,Charginginfrastructure_BEV_MEL,Chargingspeed_BEV_MEL,Homedistnearestcharger_BEV_MEL,Homechargeravailability_BEV_MEL,Purchaseprice_BEV_STOR,Operationcost_BEV_STOR,Yearlycost__BEV_STOR,Range_BEV_STOR,Acceleration_BEV_STOR,Size_BEV_STOR,CO2_BEV_STOR,Charginginfrastructure_BEV_STOR,Chargingspeed_BEV_STOR,Homedistnearestcharger_BEV_STOR,Home

In [14]:
pd.options.display.max_columns = 50

#### Utility functions

In [15]:
'''
V1 = ASC_1_RND + B_PP*Purchaseprice_ICV_MINI + B_OC*Operationcost_ICV_MINI + B_YC*Yearlycost__ICV_MINI + B_Ra*Range_ICV_MINI + B_Acc*Acceleration_ICV_MINI + B_SizeLarge*Size_ICV_MINI_Large_Dummy + B_Co2*CO2_ICV_MINI + B_ChInf*Charginginfrastructure_ICV_MINI + B_ChSpeed*Chargingspeed_ICV_MINI + B_Dist*Homedistnearestcharger_ICV_MINI + B_HomeCh*Homechargeravailability_ICV_MINI
V2 = ASC_2_RND + B_PP*Purchaseprice_ICV_LIL + B_OC*Operationcost_ICV_LIL + B_YC*Yearlycost__ICV_LIL + B_Ra*Range_ICV_LIL + B_Acc*Acceleration_ICV_LIL + B_SizeLarge*Size_ICV_LIL_Large_Dummy + B_Co2*CO2_ICV_LIL + B_ChInf*Charginginfrastructure_ICV_LIL + B_ChSpeed*Chargingspeed_ICV_LIL + B_Dist*Homedistnearestcharger_ICV_LIL + B_HomeCh*Homechargeravailability_ICV_LIL
V3 = ASC_3_RND + B_PP*Purchaseprice_ICV_MEL + B_OC*Operationcost_ICV_MEL + B_YC*Yearlycost__ICV_MEL + B_Ra*Range_ICV_MEL + B_Acc*Acceleration_ICV_MEL + B_SizeLarge*Size_ICV_MEL_Large_Dummy + B_Co2*CO2_ICV_MEL + B_ChInf*Charginginfrastructure_ICV_MEL + B_ChSpeed*Chargingspeed_ICV_MEL + B_Dist*Homedistnearestcharger_ICV_MEL + B_HomeCh*Homechargeravailability_ICV_MEL
V4 = ASC_4_RND + B_PP*Purchaseprice_ICV_STOR + B_OC*Operationcost_ICV_STOR + B_YC*Yearlycost__ICV_STOR + B_Ra*Range_ICV_STOR + B_Acc*Acceleration_ICV_STOR + B_SizeLarge*Size_ICV_STOR_Large_Dummy + B_Co2*CO2_ICV_STOR + B_ChInf*Charginginfrastructure_ICV_STOR + B_ChSpeed*Chargingspeed_ICV_STOR + B_Dist*Homedistnearestcharger_ICV_STOR + B_HomeCh*Homechargeravailability_ICV_STOR
V5 = ASC_5_RND + B_PP*Purchaseprice_ICV_PREM + B_OC*Operationcost_ICV_PREM + B_YC*Yearlycost__ICV_PREM + B_Ra*Range_ICV_PREM + B_Acc*Acceleration_ICV_PREM + B_SizeLarge*Size_ICV_PREM_Large_Dummy + B_Co2*CO2_ICV_PREM + B_ChInf*Charginginfrastructure_ICV_PREM + B_ChSpeed*Chargingspeed_ICV_PREM + B_Dist*Homedistnearestcharger_ICV_PREM + B_HomeCh*Homechargeravailability_ICV_PREM
V6 = ASC_6_RND + B_PP*Purchaseprice_ICV_LUK + B_OC*Operationcost_ICV_LUK + B_YC*Yearlycost__ICV_LUK + B_Ra*Range_ICV_LUK + B_Acc*Acceleration_ICV_LUK + B_SizeLarge*Size_ICV_LUK_Large_Dummy + B_Co2*CO2_ICV_LUK + B_ChInf*Charginginfrastructure_ICV_LUK + B_ChSpeed*Chargingspeed_ICV_LUK + B_Dist*Homedistnearestcharger_ICV_LUK + B_HomeCh*Homechargeravailability_ICV_LUK

V7 = ASC_7_RND + B_PP*Purchaseprice_BEV_MINI + B_OC*Operationcost_BEV_MINI + B_YC*Yearlycost__BEV_MINI + B_Ra*Range_BEV_MINI + B_Acc*Acceleration_BEV_MINI + B_SizeLarge*Size_BEV_MINI_Large_Dummy + B_Co2*CO2_BEV_MINI + B_ChInf*Charginginfrastructure_BEV_MINI + B_ChSpeed*Chargingspeed_BEV_MINI + B_Dist*Homedistnearestcharger_BEV_MINI + B_HomeCh*Homechargeravailability_BEV_MINI
V8 = ASC_8_RND + B_PP*Purchaseprice_BEV_LIL + B_OC*Operationcost_BEV_LIL + B_YC*Yearlycost__BEV_LIL + B_Ra*Range_BEV_LIL + B_Acc*Acceleration_BEV_LIL + B_SizeLarge*Size_BEV_LIL_Large_Dummy + B_Co2*CO2_BEV_LIL + B_ChInf*Charginginfrastructure_BEV_LIL + B_ChSpeed*Chargingspeed_BEV_LIL + B_Dist*Homedistnearestcharger_BEV_LIL + B_HomeCh*Homechargeravailability_BEV_LIL
V9 = ASC_9_RND + B_PP*Purchaseprice_BEV_MEL + B_OC*Operationcost_BEV_MEL + B_YC*Yearlycost__BEV_MEL + B_Ra*Range_BEV_MEL + B_Acc*Acceleration_BEV_MEL + B_SizeLarge*Size_BEV_MEL_Large_Dummy + B_Co2*CO2_BEV_MEL + B_ChInf*Charginginfrastructure_BEV_MEL + B_ChSpeed*Chargingspeed_BEV_MEL + B_Dist*Homedistnearestcharger_BEV_MEL + B_HomeCh*Homechargeravailability_BEV_MEL
V10 = ASC_10_RND + B_PP*Purchaseprice_BEV_STOR + B_OC*Operationcost_BEV_STOR + B_YC*Yearlycost__BEV_STOR + B_Ra*Range_BEV_STOR + B_Acc*Acceleration_BEV_STOR + B_SizeLarge*Size_BEV_STOR_Large_Dummy + B_Co2*CO2_BEV_STOR + B_ChInf*Charginginfrastructure_BEV_STOR + B_ChSpeed*Chargingspeed_BEV_STOR + B_Dist*Homedistnearestcharger_BEV_STOR + B_HomeCh*Homechargeravailability_BEV_STOR
V11 = ASC_11_RND + B_PP*Purchaseprice_BEV_PREM + B_OC*Operationcost_BEV_PREM + B_YC*Yearlycost__BEV_PREM + B_Ra*Range_BEV_PREM + B_Acc*Acceleration_BEV_PREM + B_SizeLarge*Size_BEV_PREM_Large_Dummy + B_Co2*CO2_BEV_PREM + B_ChInf*Charginginfrastructure_BEV_PREM + B_ChSpeed*Chargingspeed_BEV_PREM + B_Dist*Homedistnearestcharger_BEV_PREM + B_HomeCh*Homechargeravailability_BEV_PREM
V12 = ASC_12_RND + B_PP*Purchaseprice_BEV_LUK + B_OC*Operationcost_BEV_LUK + B_YC*Yearlycost__BEV_LUK + B_Ra*Range_BEV_LUK + B_Acc*Acceleration_BEV_LUK + B_SizeLarge*Size_BEV_LUK_Large_Dummy + B_Co2*CO2_BEV_LUK + B_ChInf*Charginginfrastructure_BEV_LUK + B_ChSpeed*Chargingspeed_BEV_LUK + B_Dist*Homedistnearestcharger_BEV_LUK + B_HomeCh*Homechargeravailability_BEV_LUK

V13 = ASC_13_RND + B_PP*Purchaseprice_PHEV_MINI + B_OC*Operationcost_PHEV_MINI + B_YC*Yearlycost__PHEV_MINI + B_Ra*Range_PHEV_MINI + B_Acc*Acceleration_PHEV_MINI + B_SizeLarge*Size_PHEV_MINI_Large_Dummy + B_Co2*CO2_PHEV_MINI + B_ChInf*Charginginfrastructure_PHEV_MINI + B_ChSpeed*Chargingspeed_PHEV_MINI + B_Dist*Homedistnearestcharger_PHEV_MINI + B_HomeCh*Homechargeravailability_PHEV_MIN
V14 = ASC_14_RND + B_PP*Purchaseprice_PHEV_LIL + B_OC*Operationcost_PHEV_LIL + B_YC*Yearlycost__PHEV_LIL + B_Ra*Range_PHEV_LIL + B_Acc*Acceleration_PHEV_LIL + B_SizeLarge*Size_PHEV_LIL_Large_Dummy + B_Co2*CO2_PHEV_LIL + B_ChInf*Charginginfrastructure_PHEV_LIL + B_ChSpeed*Chargingspeed_PHEV_LIL + B_Dist*Homedistnearestcharger_PHEV_LIL + B_HomeCh*Homechargeravailability_PHEV_LIL
V15 = ASC_15_RND + B_PP*Purchaseprice_PHEV_MEL + B_OC*Operationcost_PHEV_MEL + B_YC*Yearlycost__PHEV_MEL + B_Ra*Range_PHEV_MEL + B_Acc*Acceleration_PHEV_MEL + B_SizeLarge*Size_PHEV_MEL_Large_Dummy + B_Co2*CO2_PHEV_MEL + B_ChInf*Charginginfrastructure_PHEV_MEL + B_ChSpeed*Chargingspeed_PHEV_MEL + B_Dist*Homedistnearestcharger_PHEV_MEL + B_HomeCh*Homechargeravailability_PHEV_MEL
V16 = ASC_16_RND + B_PP*Purchaseprice_PHEV_STOR + B_OC*Operationcost_PHEV_STOR + B_YC*Yearlycost__PHEV_STOR + B_Ra*Range_PHEV_STOR + B_Acc*Acceleration_PHEV_STOR + B_SizeLarge*Size_PHEV_STOR_Large_Dummy + B_Co2*CO2_PHEV_STOR + B_ChInf*Charginginfrastructure_PHEV_STOR + B_ChSpeed*Chargingspeed_PHEV_STOR + B_Dist*Homedistnearestcharger_PHEV_STOR + B_HomeCh*Homechargeravailability_PHEV_STO
V17 = ASC_17_RND + B_PP*Purchaseprice_PHEV_PREM + B_OC*Operationcost_PHEV_PREM + B_YC*Yearlycost__PHEV_PREM + B_Ra*Range_PHEV_PREM + B_Acc*Acceleration_PHEV_PREM + B_SizeLarge*Size_PHEV_PREM_Large_Dummy + B_Co2*CO2_PHEV_PREM + B_ChInf*Charginginfrastructure_PHEV_PREM + B_ChSpeed*Chargingspeed_PHEV_PREM + B_Dist*Homedistnearestcharger_PHEV_PREM + B_HomeCh*Homechargeravailability_PHEV_PRE
V18 = ASC_18 + B_PP*Purchaseprice_PHEV_LUK + B_OC*Operationcost_PHEV_LUK + B_YC*Yearlycost__PHEV_LUK + B_Ra*Range_PHEV_LUK + B_Acc*Acceleration_PHEV_LUK + B_SizeLarge*Size_PHEV_LUK_Large_Dummy + B_Co2*CO2_PHEV_LUK + B_ChInf*Charginginfrastructure_PHEV_LUK + B_ChSpeed*Chargingspeed_PHEV_LUK + B_Dist*Homedistnearestcharger_PHEV_LUK + B_HomeCh*Homechargeravailability_PHEV_LUK
''';

Parameters

In [16]:
ASC_1 = -6.17
ASC_1_S = 2.86
ASC_2 = -6.51
ASC_2_S = 3.27
ASC_3 = -5.41
ASC_3_S = 2.81
ASC_4 = -5.9
ASC_4_S = 2.86
ASC_5 = -5.67
ASC_5_S = 3.11
ASC_6 = -5.92
ASC_6_S = 5.38
ASC_7 = -5.57
ASC_7_S = 2.34
ASC_8 = -6.15
ASC_8_S = 2.04
ASC_9 = -5.01
ASC_9_S = 2.02
ASC_10 = -5.12
ASC_10_S = 2.04
ASC_11 = -5.02
ASC_11_S = 2.46
ASC_12 = -3.14
ASC_12_S = 2.37
ASC_13 = -4.7
ASC_13_S = 2.29
ASC_14 = -5.29
ASC_14_S = 2.75
ASC_15 = -3.52
ASC_15_S = 2.73
ASC_16 = -3.59
ASC_16_S = 2.32
ASC_17 = -3.88
ASC_17_S = 2.6

B_Acc = -0.0345
B_ChInf = -0.000261
B_ChSpeed = 0.0039
B_Co2 = -0.00325
B_Dist = -0.00106
B_HomeCh = 0.137
B_OC = -0.608
B_PP = -6.5 * 10 ** -6
B_Ra = 0.0019
B_SizeLarge = 0.23
B_YC = -9.55 * 10 ** -5

Utility functions ICEV

In [17]:
def V1(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_1+ASC_1_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V2(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_2+ASC_2_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V3(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_3+ASC_3_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V4(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_4+ASC_4_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V5(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_5+ASC_5_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V6(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_6+ASC_6_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

Utility functions BEV

In [18]:
def V7(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_7+ASC_7_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V8(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_8+ASC_8_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V9(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_9+ASC_9_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V10(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_10+ASC_10_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V11(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_11+ASC_11_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V12(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_12+ASC_12_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

Utility functions PHEV

In [19]:
def V13(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_13+ASC_13_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V14(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_14+ASC_14_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V15(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_15+ASC_15_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V16(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_16+ASC_16_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V17(Rand,Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return ASC_17+ASC_17_S*Rand + B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

def V18(Purchaseprice,Operationcost,Yearlycost,Range,Acceleration,Size_Large_Dummy,CO2,Charginginfrastructure,Chargingspeed,Homedistnearestcharger,Homechargeravailability):
    return B_PP*Purchaseprice + B_OC*Operationcost + B_YC*Yearlycost + B_Ra*Range + B_Acc*Acceleration + B_SizeLarge*Size_Large_Dummy + B_Co2*CO2 + B_ChInf*Charginginfrastructure + B_ChSpeed*Chargingspeed + B_Dist*Homedistnearestcharger + B_HomeCh*Homechargeravailability

#### Probability function

In [22]:
def prob(mu, *args):
    total = 0.0
    for arg in args:
        if arg[0] == 1:
            total += np.exp(arg[1] * mu)
    prob = []
    for arg in args:
        if arg[0] == 1:
            prob.append(np.exp(mu*arg[1]) / total)
        else:
            prob.append(0)
    return prob

#### Mapping

In [23]:
mapping = {
    0:"ICEV_1",1:"ICEV_2",2:"ICEV_3",3:"ICEV_4",4:"ICEV_5",5:"ICEV_6",
    6:"BEV_1",7:"BEV_2",8:"BEV_3",9:"BEV_4",10:"BEV_5",11:"BEV_6",
    12:"PHEV_1",13:"PHEV_2",14:"PHEV_3",15:"PHEV_4",16:"PHEV_5",17:"PHEV_6"
}

#### Computing market share estimates

Calculating probabilities

In [24]:
def ProbMatrix():
    mu = 1
    probMatrix = []
    for index, row in df_mod.iterrows():
        v1 = (row['AV1'], V1(np.random.normal(),row['Purchaseprice_ICV_MINI'],row['Operationcost_ICV_MINI'],row['Yearlycost__ICV_MINI'],row['Range_ICV_MINI'],row['Acceleration_ICV_MINI'],row['Size_ICV1'],row['CO2_ICV_MINI'],row['Charginginfrastructure_ICV_MINI'],row['Chargingspeed_ICV_MINI'],row['Homedistnearestcharger_ICV_MINI'],row['Homechargeravailability_ICV_MINI']))
        v2 = (row['AV2'], V2(np.random.normal(),row['Purchaseprice_ICV_LIL'],row['Operationcost_ICV_LIL'],row['Yearlycost__ICV_LIL'],row['Range_ICV_LIL'],row['Acceleration_ICV_LIL'],row['Size_ICV_LIL'],row['CO2_ICV_LIL'],row['Charginginfrastructure_ICV_LIL'],row['Chargingspeed_ICV_LIL'],row['Homedistnearestcharger_ICV_LIL'],row['Homechargeravailability_ICV_LIL']))
        v3 = (row['AV3'], V3(np.random.normal(),row['Purchaseprice_ICV_MEL'],row['Operationcost_ICV_MEL'],row['Yearlycost__ICV_MEL'],row['Range_ICV_MEL'],row['Acceleration_ICV_MEL'],row['Size_ICV_MEL'],row['CO2_ICV_MEL'],row['Charginginfrastructure_ICV_MEL'],row['Chargingspeed_ICV_MEL'],row['Homedistnearestcharger_ICV_MEL'],row['Homechargeravailability_ICV_MEL']))
        v4 = (row['AV4'], V4(np.random.normal(),row['Purchaseprice_ICV_STOR'],row['Operationcost_ICV_STOR'],row['Yearlycost__ICV_STOR'],row['Range_ICV_STOR'],row['Acceleration_ICV_STOR'],row['Size_ICV_STOR'],row['CO2_ICV_STOR'],row['Charginginfrastructure_ICV_STOR'],row['Chargingspeed_ICV_STOR'],row['Homedistnearestcharger_ICV_STOR'],row['Homechargeravailability_ICV_STOR']))
        v5 = (row['AV5'], V5(np.random.normal(),row['Purchaseprice_ICV_PREM'],row['Operationcost_ICV_PREM'],row['Yearlycost__ICV_PREM'],row['Range_ICV_PREM'],row['Acceleration_ICV_PREM'],row['Size_ICV_PREM'],row['CO2_ICV_PREM'],row['Charginginfrastructure_ICV_PREM'],row['Chargingspeed_ICV_PREM'],row['Homedistnearestcharger_ICV_PREM'],row['Homechargeravailability_ICV_PREM']))
        v6 = (row['AV6'], V6(np.random.normal(),row['Purchaseprice_ICV_LUK'],row['Operationcost_ICV_LUK'],row['Yearlycost__ICV_LUK'],row['Range_ICV_LUK'],row['Acceleration_ICV_LUK'],row['Size_ICV_LUK'],row['CO2_ICV_LUK'],row['Charginginfrastructure_ICV_LUK'],row['Chargingspeed_ICV_LUK'],row['Homedistnearestcharger_ICV_LUK'],row['Homechargeravailability_ICV_LUK']))

        v7 = (row['AV7'], V7(np.random.normal(),row['Purchaseprice_BEV_MINI'],row['Operationcost_BEV_MINI'],row['Yearlycost__BEV_MINI'],row['Range_BEV_MINI'],row['Acceleration_BEV_MINI'],row['Size_BEV_MINI'],row['CO2_BEV_MINI'],row['Charginginfrastructure_BEV_MINI'],row['Chargingspeed_BEV_MINI'],row['Homedistnearestcharger_BEV_MINI'],row['Homechargeravailability_BEV_MINI']))
        v8 = (row['AV8'], V8(np.random.normal(),row['Purchaseprice_BEV_LIL'],row['Operationcost_BEV_LIL'],row['Yearlycost__BEV_LIL'],row['Range_BEV_LIL'],row['Acceleration_BEV_LIL'],row['Size_BEV_LIL'],row['CO2_BEV_LIL'],row['Charginginfrastructure_BEV_LIL'],row['Chargingspeed_BEV_LIL'],row['Homedistnearestcharger_BEV_LIL'],row['Homechargeravailability_BEV_LIL']))
        v9 = (row['AV9'], V9(np.random.normal(),row['Purchaseprice_BEV_MEL'],row['Operationcost_BEV_MEL'],row['Yearlycost__BEV_MEL'],row['Range_BEV_MEL'],row['Acceleration_BEV_MEL'],row['Size_BEV_MEL'],row['CO2_BEV_MEL'],row['Charginginfrastructure_BEV_MEL'],row['Chargingspeed_BEV_MEL'],row['Homedistnearestcharger_BEV_MEL'],row['Homechargeravailability_BEV_MEL']))
        v10 = (row['AV10'], V10(np.random.normal(),row['Purchaseprice_BEV_STOR'],row['Operationcost_BEV_STOR'],row['Yearlycost__BEV_STOR'],row['Range_BEV_STOR'],row['Acceleration_BEV_STOR'],row['Size_BEV_STOR'],row['CO2_BEV_STOR'],row['Charginginfrastructure_BEV_STOR'],row['Chargingspeed_BEV_STOR'],row['Homedistnearestcharger_BEV_STOR'],row['Homechargeravailability_BEV_STOR']))
        v11 = (row['AV11'], V11(np.random.normal(),row['Purchaseprice_BEV_PREM'],row['Operationcost_BEV_PREM'],row['Yearlycost__BEV_PREM'],row['Range_BEV_PREM'],row['Acceleration_BEV_PREM'],row['Size_BEV_PREM'],row['CO2_BEV_PREM'],row['Charginginfrastructure_BEV_PREM'],row['Chargingspeed_BEV_PREM'],row['Homedistnearestcharger_BEV_PREM'],row['Homechargeravailability_BEV_PREM']))
        v12 = (row['AV12'], V12(np.random.normal(),row['Purchaseprice_BEV_LUK'],row['Operationcost_BEV_LUK'],row['Yearlycost__BEV_LUK'],row['Range_BEV_LUK'],row['Acceleration_BEV_LUK'],row['Size_BEV_LUK'],row['CO2_BEV_LUK'],row['Charginginfrastructure_BEV_LUK'],row['Chargingspeed_BEV_LUK'],row['Homedistnearestcharger_BEV_LUK'],row['Homechargeravailability_BEV_LUK']))

        v13 = (row['AV13'], V13(np.random.normal(),row['Purchaseprice_PHEV_MINI'],row['Operationcost_PHEV_MINI'],row['Yearlycost__PHEV_MINI'],row['Range_PHEV_MINI'],row['Acceleration_PHEV_MINI'],row['Size_PHEV_MINI'],row['CO2_PHEV_MINI'],row['Charginginfrastructure_PHEV_MINI'],row['Chargingspeed_PHEV_MINI'],row['Homedistnearestcharger_PHEV_MINI'],row['Homechargeravailability_PHEV_MIN']))
        v14 = (row['AV14'], V14(np.random.normal(),row['Purchaseprice_PHEV_LIL'],row['Operationcost_PHEV_LIL'],row['Yearlycost__PHEV_LIL'],row['Range_PHEV_LIL'],row['Acceleration_PHEV_LIL'],row['Size_PHEV_LIL'],row['CO2_PHEV_LIL'],row['Charginginfrastructure_PHEV_LIL'],row['Chargingspeed_PHEV_LIL'],row['Homedistnearestcharger_PHEV_LIL'],row['Homechargeravailability_PHEV_LIL']))
        v15 = (row['AV15'], V15(np.random.normal(),row['Purchaseprice_PHEV_MEL'],row['Operationcost_PHEV_MEL'],row['Yearlycost__PHEV_MEL'],row['Range_PHEV_MEL'],row['Acceleration_PHEV_MEL'],row['Size_PHEV_MEL'],row['CO2_PHEV_MEL'],row['Charginginfrastructure_PHEV_MEL'],row['Chargingspeed_PHEV_MEL'],row['Homedistnearestcharger_PHEV_MEL'],row['Homechargeravailability_PHEV_MEL']))
        v16 = (row['AV16'], V16(np.random.normal(),row['Purchaseprice_PHEV_STOR'],row['Operationcost_PHEV_STOR'],row['Yearlycost__PHEV_STOR'],row['Range_PHEV_STOR'],row['Acceleration_PHEV_STOR'],row['Size_PHEV_STOR'],row['CO2_PHEV_STOR'],row['Charginginfrastructure_PHEV_STOR'],row['Chargingspeed_PHEV_STOR'],row['Homedistnearestcharger_PHEV_STOR'],row['Homechargeravailability_PHEV_STO']))   
        v17 = (row['AV17'], V17(np.random.normal(),row['Purchaseprice_PHEV_PREM'],row['Operationcost_PHEV_PREM'],row['Yearlycost__PHEV_PREM'],row['Range_PHEV_PREM'],row['Acceleration_PHEV_PREM'],row['Size_PHEV_PREM'],row['CO2_PHEV_PREM'],row['Charginginfrastructure_PHEV_PREM'],row['Chargingspeed_PHEV_PREM'],row['Homedistnearestcharger_PHEV_PREM'],row['Homechargeravailability_PHEV_PRE']))
        v18 = (row['AV18'], V18(row['Purchaseprice_PHEV_LUK'],row['Operationcost_PHEV_LUK'],row['Yearlycost__PHEV_LUK'],row['Range_PHEV_LUK'],row['Acceleration_PHEV_LUK'],row['Size_PHEV_LUK'],row['CO2_PHEV_LUK'],row['Charginginfrastructure_PHEV_LUK'],row['Chargingspeed_PHEV_LUK'],row['Homedistnearestcharger_PHEV_LUK'],row['Homechargeravailability_PHEV_LUK']))


        probVector = prob(mu, v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18)
        probMatrix.append(probVector)
    
    return probMatrix

In [25]:
resultMatrix = []
runs = 10
for i in range(runs):
    resultMatrix.append(ProbMatrix())

Sample enumeration

In [28]:
aux_a = []
for i in range(runs):
    aux_b = []
    for j in range(len(mapping)):
        aux_b.append((sum([x[j] for x in resultMatrix[i]]) / len(resultMatrix[i])))
    aux_a.append(aux_b)

sampleEnum = {}
for i in range(len(mapping)):
    sampleEnum[mapping[i]] = round((sum([x[i] for x in aux_a]) / len(aux_a)) * 100,3)
df_res = pd.DataFrame([sampleEnum])
df_res.style.hide_index()                          

ICEV_1,ICEV_2,ICEV_3,ICEV_4,ICEV_5,ICEV_6,BEV_1,BEV_2,BEV_3,BEV_4,BEV_5,BEV_6,PHEV_1,PHEV_2,PHEV_3,PHEV_4,PHEV_5,PHEV_6
6.349000,9.217000,15.104000,6.654000,3.594000,0.827000,5.800000,6.276000,16.059000,9.461000,3.694000,0.696000,2.052000,2.666000,7.297000,3.090000,0.887000,0.274000


#### Elasticities

In [41]:
col = df_mod.columns.to_list()
col_pp = []
for c in col:
    if Search("Purchaseprice_...*_",c):
        col_pp.append(c)
#col_pp

In [42]:
means = df_res.transpose()[0].tolist()
means

[6.349,
 9.217,
 15.104,
 6.654,
 3.594,
 0.827,
 5.8,
 6.276,
 16.059,
 9.461,
 3.694,
 0.696,
 2.052,
 2.666,
 7.297,
 3.09,
 0.887,
 0.274]

In [43]:
#elasticities = []
for i,c in enumerate(col_pp):
    print(c,round(B_PP * df_mod[c].mean() * (1 -  means[i]),3))

Purchaseprice_ICV_MINI 0.935
Purchaseprice_ICV_LIL 3.861
Purchaseprice_ICV_MEL 14.437
Purchaseprice_ICV_STOR 7.182
Purchaseprice_ICV_PREM 1.92
Purchaseprice_ICV_LUK -0.159
Purchaseprice_BEV_MINI 0.885
Purchaseprice_BEV_LIL 2.972
Purchaseprice_BEV_MEL 15.53
Purchaseprice_BEV_STOR 9.916
Purchaseprice_BEV_PREM 2.071
Purchaseprice_BEV_LUK -0.28
Purchaseprice_PHEV_MINI 0.1
Purchaseprice_PHEV_LIL 0.424
Purchaseprice_PHEV_MEL 3.271
Purchaseprice_PHEV_STOR 1.415
Purchaseprice_PHEV_PREM -0.052
Purchaseprice_PHEV_LUK -0.336
